In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.utilities import get_engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
engine = get_engine()

In [20]:
# Get list of stock and its start date
stock_query = f"""
    SELECT 
        stock_code,
        company_name,
        first_transaction_date
    FROM public.stock_info
    WHERE
        stock_code = 'BVH'
"""
df = pd.read_sql_query(stock_query, engine)

In [25]:
def calculate_profit(stocks : pd.DataFrame):
    result = pd.DataFrame(columns = ['stock_code', 'buy_today', 'profit'])
    today = datetime.today().date()
    for _, stock in stocks.iterrows():
        query = f"""
            SELECT * 
            FROM public.transaction
            WHERE
                date >= DATE '{stock["first_transaction_date"]}'
                AND
                date <= DATE '{today}'
                AND
                stock_code = '{stock["stock_code"]}'
        """
        df = pd.read_sql_query(query, engine)
        df['MA10'] = df['close_price'].rolling(10).mean()
        df['MA50'] = df['close_price'].rolling(50).mean()
        df = df.dropna()
        df['shares'] = [1 if df.loc[i, 'MA10'] > df.loc[i, 'MA50'] else 0 for i in df.index]
        df['future_close_price'] = df['close_price'].shift(-1)
        df['profit'] = [df.loc[i, 'future_close_price'] - df.loc[i, 'close_price'] for i in df.index]
        df['wealth'] = df['profit'].cumsum()
        if len(df) >= 2:
            result.loc[len(result)] = [stock["stock_code"], df['shares'].iloc[-1], df['wealth'].iloc[-2]]
            result.index = result.index + 1

    return result


In [26]:
result = calculate_profit(df)

In [10]:
result.index = result['stock_code']
result = result.drop('stock_code', axis = 1)

In [27]:
result

,stock_code,buy_today,profit
1,BVH,1,-1.0
